# Dopravní problém v lineárním programování

Projdeme si klasický **dopravní problém**:

- máme několik skladů (dodavatelů) s danými kapacitami,
- několik odběratelů (měst) s danými požadavky,
- a známé jednotkové náklady dopravy mezi každým skladem a každým odběratelem.

Cílem je **nalézt takové rozdělení přepravovaných množství**, aby byly splněny všechny požadavky a kapacity a **celkové náklady byly minimální**.

---

## Obecná matematická formulace

Označme:

- $i = 1,\dots,n$ index dodavatelů,
- $j = 1,\dots,m$ index odběratelů,
- $a_i$ kapacita $i$-tého dodavatele,
- $b_j$ požadavek $j$-tého odběratele,
- $c_{ij}$ náklady na přepravu jedné jednotky z dodavatele $i$ k odběrateli $j$,
- rozhodovací proměnná $x_{ij} \ge 0$ = množství přepravené z dodavatele $i$ k odběrateli $j$.

Minimalizujeme celkové náklady

$$
\min z = \sum_{i=1}^{n} \sum_{j=1}^{m} c_{ij} x_{ij}
$$

při omezeních:

1. **Kapacity dodavatelů**

$$
\sum_{j=1}^{m} x_{ij} = a_i, \quad i = 1,\dots,n
$$

2. **Požadavky odběratelů**

$$
\sum_{i=1}^{n} x_{ij} = b_j, \quad j = 1,\dots,m
$$

3. **Nezápornost**

$$
x_{ij} \ge 0 \quad \text{pro všechna } i,j.
$$

V našem konkrétním příkladu budou celkové kapacity a celkové požadavky stejné, takže tzv. *vyrovnaný dopravní problém* – proto lze oba typy omezení psát jako rovnosti.


In [ ]:
# Depends on whether PuLP is installed in your environment, comment/uncomment and run this cell once:
!pip install pulp


## Konkrétní příklad – sklady a města v ČR

Uvažujme následující situaci:

- 3 sklady (dodavatelé): **Liberec, Olomouc, Plzeň**
- 4 města (odběratelé): **Brno, Praha, Ostrava, Zlín**

Máme tabulku vzdáleností, resp. jednotkových nákladů dopravy (čím větší číslo, tím dražší přeprava):

| Vzdálenost / Náklady | Brno | Praha | Ostrava | Zlín | Kapacity |
|----------------------|:----:|:-----:|:-------:|:----:|:--------:|
| **Liberec**          | 307  | 109   | 440     | 400  | 6000     |
| **Olomouc**          | 77   | 282   | 98      | 63   | 2000     |
| **Plzeň**            | 296  | 91    | 463     | 389  | 3000     |
| **Požadavky**        | 3000 | 5000  | 2000    | 1000 |          |

Interpretace:

- číslo 307 znamená náklad na přepravu jedné jednotky z Liberce do Brna,
- kapacita 6000 znamená, že z Liberce lze maximálně odeslat 6000 jednotek,
- požadavek Brna 3000 znamená, že Brno potřebuje přesně 3000 jednotek.

Součet kapacit je $6000 + 2000 + 3000 = 11000$ a součet požadavků je $3000 + 5000 + 2000 + 1000 = 11000$, takže problém je opravdu vyrovnaný.

V následující buňce data převedeme do jednoduchých Python strukturovaných proměnných.


In [ ]:
from itertools import product

# --- Basic sets (names of suppliers and customers) ---
suppliers = ["Liberec", "Olomouc", "Plzeň"]
customers = ["Brno", "Praha", "Ostrava", "Zlín"]

# --- Cost matrix C[i][j] ---
# Row order follows 'suppliers', column order follows 'customers'.
C = [
    [307, 109, 440, 400],  # Liberec -> Brno, Praha, Ostrava, Zlín
    [77,  282, 98,  63 ],  # Olomouc -> Brno, Praha, Ostrava, Zlín
    [296, 91,  463, 389],  # Plzeň   -> Brno, Praha, Ostrava, Zlín
]

# --- Supply capacities for each supplier ---
supply = [6000, 2000, 3000]

# --- Demand requirements for each customer ---
demand = [3000, 5000, 2000, 1000]

# --- Convert everything into dictionaries keyed by names (more convenient with PuLP) ---
costs = {
    (s, c): C[i][j]
    for i, s in enumerate(suppliers)
    for j, c in enumerate(customers)
}

supply_dict = {s: supply[i] for i, s in enumerate(suppliers)}
demand_dict = {c: demand[j] for j, c in enumerate(customers)}

print("Suppliers:", suppliers)
print("Customers:", customers)
print("Total supply:", sum(supply_dict.values()))
print("Total demand:", sum(demand_dict.values()))


Suppliers: ['Liberec', 'Olomouc', 'Plzeň']
Customers: ['Brno', 'Praha', 'Ostrava', 'Zlín']
Total supply: 11000
Total demand: 11000


## Vytvoření modelu v PuLP

Teď zavedeme rozhodovací proměnné a sestavíme model:

- proměnná $x_{ij}$ bude reprezentována jako `x[s][c]` (kolik jednotek pošleme z dodavatele `s` do odběratele `c`),
- všechny proměnné budou **spojité a nezáporné**,
- cílem je minimalizovat `sum(costs[(s, c)] * x[s][c])` přes všechny dvojice (dodavatel, odběratel),
- kapacitní omezení: součet z daného skladu `s` do všech měst se musí rovnat jeho kapacitě,
- požadavky měst: součet do daného města `c` ze všech skladů se musí rovnat jeho požadavku.

Protože je problém vyrovnaný (celková kapacita = celkový požadavek), používáme **rovnosti**. V nevyrovnaném případě se běžně používají nerovnosti `<=` u kapacit a `>=` u požadavků a přidávají se tzv. fiktivní dodavatelé/odběratelé.


In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

# --- Create the optimization model ---
model = LpProblem(name="Dopravni_problem", sense=LpMinimize)

# --- Decision variables: x[s][c] >= 0, continuous by default ---
# x[s][c] is the amount shipped from supplier s to customer c.
x = LpVariable.dicts(
    "x",
    (suppliers, customers),
    lowBound=0  # no upper bound, only capacity and demand constraints will limit it
    # cat is Continuous by default
)

# --- Objective function: minimize total transport cost ---
model += lpSum(
    x[s][c] * costs[(s, c)]
    for s, c in product(suppliers, customers)
), "Total_transport_cost"

# --- Supply constraints: each supplier ships exactly its capacity ---
for s in suppliers:
    model += lpSum(x[s][c] for c in customers) == supply_dict[s], f"Supply_{s}"

# --- Demand constraints: each customer receives exactly its demand ---
for c in customers:
    model += lpSum(x[s][c] for s in suppliers) == demand_dict[c], f"Demand_{c}"

# Optional: print a human-readable representation of the model
print(model)


Dopravni_problem:
MINIMIZE
307*x_Liberec_Brno + 440*x_Liberec_Ostrava + 109*x_Liberec_Praha + 400*x_Liberec_Zlín + 77*x_Olomouc_Brno + 98*x_Olomouc_Ostrava + 282*x_Olomouc_Praha + 63*x_Olomouc_Zlín + 296*x_Plzeň_Brno + 463*x_Plzeň_Ostrava + 91*x_Plzeň_Praha + 389*x_Plzeň_Zlín + 0.0
SUBJECT TO
Supply_Liberec: x_Liberec_Brno + x_Liberec_Ostrava + x_Liberec_Praha
 + x_Liberec_Zlín = 6000

Supply_Olomouc: x_Olomouc_Brno + x_Olomouc_Ostrava + x_Olomouc_Praha
 + x_Olomouc_Zlín = 2000

Supply_Plzeň: x_Plzeň_Brno + x_Plzeň_Ostrava + x_Plzeň_Praha + x_Plzeň_Zlín
 = 3000

Demand_Brno: x_Liberec_Brno + x_Olomouc_Brno + x_Plzeň_Brno = 3000

Demand_Praha: x_Liberec_Praha + x_Olomouc_Praha + x_Plzeň_Praha = 5000

Demand_Ostrava: x_Liberec_Ostrava + x_Olomouc_Ostrava + x_Plzeň_Ostrava = 2000

Demand_Zlín: x_Liberec_Zlín + x_Olomouc_Zlín + x_Plzeň_Zlín = 1000

VARIABLES
x_Liberec_Brno Continuous
x_Liberec_Ostrava Continuous
x_Liberec_Praha Continuous
x_Liberec_Zlín Continuous
x_Olomouc_Brno Continuous

## Řešení modelu a interpretace výsledků

Nyní model předáme výchozímu solveru PuLP:

1. **Status** – zda bylo nalezeno optimální řešení.
2. **Hodnota účelové funkce** – minimální celkové náklady.
3. **Hodnoty proměnných** $x_{ij}$ – kolik jednotek se má skutečně posílat po jednotlivých trasách.
4. Pro kontrolu si můžeme spočítat, kolik se celkem odesílá z každého skladu a přijímá v každém městě.


In [ ]:
# --- Solve the model ---
result_status = model.solve()

print(f"Solver status: {LpStatus[result_status]}")
print(f"Optimální celkové náklady: {value(model.objective):,.0f}")

# --- Extract optimal shipments into a plain dictionary ---
shipments = {
    (s, c): x[s][c].value()
    for s in suppliers
    for c in customers
}

# --- Pretty-print shipment table ---
print("\nOptimální množství přepravy (v jednotkách):")
header = "{:<10}".format("") + "".join(f"{c:>10}" for c in customers)
print(header)
print("-" * len(header))

for s in suppliers:
    row = "{:<10}".format(s)
    for c in customers:
        qty = shipments[(s, c)]
        row += f"{qty:10.0f}"
    print(row)

# --- Check supply usage for each supplier ---
print("\nKontrola kapacit dodavatelů:")
for s in suppliers:
    total_out = sum(shipments[(s, c)] for c in customers)
    print(f"{s:10}: odesláno {total_out:5.0f} / kapacita {supply_dict[s]:5.0f}")

# --- Check demand satisfaction for each customer ---
print("\nKontrola požadavků odběratelů:")
for c in customers:
    total_in = sum(shipments[(s, c)] for s in suppliers)
    print(f"{c:10}: přijato {total_in:5.0f} / požadavek {demand_dict[c]:5.0f}")

# --- (Optional) print numerical value of each constraint for debugging ---
print("\nHodnoty omezení (měly by být nulové, protože jsou psané jako rovnosti):")
for name, constraint in model.constraints.items():
    # constraint.value() gives left-hand side minus right-hand side
    print(f"{name:20}: {constraint.value():6.0f}")


Solver status: Optimal
Optimální celkové náklady: 2,008,000

Optimální množství přepravy (v jednotkách):
                Brno     Praha   Ostrava      Zlín
--------------------------------------------------
Liberec         3000      2000         0      1000
Olomouc            0         0      2000         0
Plzeň              0      3000         0         0

Kontrola kapacit dodavatelů:
Liberec   : odesláno  6000 / kapacita  6000
Olomouc   : odesláno  2000 / kapacita  2000
Plzeň     : odesláno  3000 / kapacita  3000

Kontrola požadavků odběratelů:
Brno      : přijato  3000 / požadavek  3000
Praha     : přijato  5000 / požadavek  5000
Ostrava   : přijato  2000 / požadavek  2000
Zlín      : přijato  1000 / požadavek  1000

Hodnoty omezení (měly by být nulové, protože jsou psané jako rovnosti):
Supply_Liberec      :      0
Supply_Olomouc      :      0
Supply_Plzeň        :      0
Demand_Brno         :      0
Demand_Praha        :      0
Demand_Ostrava      :      0
Demand_Zlín         :  

# Dvouúrovňový dopravní problém (producenti → sklady → odběratelé)

V této části rozšíříme předchozí jednostupňový dopravní problém o **meziúroveň skladů**:

- několik **producentů** (výrobních závodů),
- dva **sklady** (v našich datech města **Kladno** a **Pardubice**),
- několik **odběratelů** (Brno, Praha, Ostrava, Zlín).

Zboží se může pohybovat pouze ve směru:

$$
\text{producent} \;\Rightarrow\; \text{sklad} \;\Rightarrow\; \text{odběratel}.
$$

Cílem je opět **minimalizovat celkové dopravní náklady**, které předpokládáme úměrné vzdálenostem mezi městy (data máme v souboru `cz.csv`).

Použijeme:

- stejné **kapacity producentů** jako v předchozí úloze  
  (Liberec 6000, Olomouc 2000, Plzeň 3000),
- stejné **požadavky odběratelů**  
  (Brno 3000, Praha 5000, Ostrava 2000, Zlín 1000),
- dva sklady **Kladno** a **Pardubice** s kapacitami **7000 jednotek každý**.

Níže krok za krokem zformulujeme matematický model a implementujeme ho v PuLP.


In [ ]:
# Imports and basic setup
!pip install pulp pandas

import pandas as pd
from itertools import product
from pulp import (
    LpProblem,
    LpMinimize,
    LpVariable,
    LpStatus,
    value,
    lpSum,
)


## Krok 1 – Načtení dat a definice soustav uzlů

### 1.1 Města a role v modelu

Budeme uvažovat tyto skupiny uzlů:

- **Producenti** (výrobní závody):  
  - Liberec  
  - Olomouc  
  - Plzeň  

- **Sklady** (mezilehlé uzly):  
  - Kladno  
  - Pardubice  

- **Odběratelé** (koneční zákazníci):  
  - Brno  
  - Praha  
  - Ostrava  
  - Zlín  

### 1.2 Kapacity a požadavky

Kapacity producentů (shodné s předchozí úlohou):

| Producent | Kapacita |
|----------|----------|
| Liberec  | 6000 |
| Olomouc  | 2000 |
| Plzeň    | 3000 |

Kapacity skladů:

| Sklad    | Kapacita |
|----------|----------|
| Kladno   | 7000 |
| Pardubice | 7000 |

Požadavky odběratelů:

| Odběratel | Požadavek |
|-----------|-----------|
| Brno      | 3000 |
| Praha     | 5000 |
| Ostrava   | 2000 |
| Zlín      | 1000 |

### 1.3 Vzdálenosti mezi městy (soubor `cz.csv`)

Soubor `cz.csv` obsahuje matici vzdáleností mezi českými městy. Můžete ho stáhnout zde: https://www.dobracesta.com/tabulka-vzdalenosti-mezi-mesty/cz/  
Z něj potřebujeme:

- vzdálenosti **producent → sklad** (např. Liberec → Kladno),
- vzdálenosti **sklad → odběratel** (např. Pardubice → Brno).

Předpokládáme, že soubor `cz.csv` leží ve stejném adresáři jako tento notebook.
V následující buňce ho načteme a z něj vytvoříme odpovídající nákladové matice.


In [ ]:
# -----------------------------
# Step 1: Load CSV distances and define sets
# -----------------------------

# Load distance table from CSV
distance_df = pd.read_csv("cz.csv")

# Keep only rows that really contain city names (non-empty first column)
distance_df = distance_df[distance_df["Vzdálenost (kilometry)"].notna()]

# Use city names as index
distance_df.set_index("Vzdálenost (kilometry)", inplace=True)

# Define sets of nodes
producers = ["Liberec", "Olomouc", "Plzeň"]
warehouses = ["Kladno", "Pardubice"]
customers = ["Brno", "Praha", "Ostrava", "Zlín"]

# Supply capacities of producers
producer_supply = {
    "Liberec": 6000,
    "Olomouc": 2000,
    "Plzeň":   3000,
}

# Demand requirements of customers
customer_demand = {
    "Brno":    3000,
    "Praha":   5000,
    "Ostrava": 2000,
    "Zlín":    1000,
}

# Capacities of warehouses
warehouse_capacity = {
    "Kladno":     7000,
    "Pardubice":  7000,
}

def distance(city_from: str, city_to: str) -> float:
    """Return distance (cost) between two cities using the CSV matrix."""
    return float(distance_df.loc[city_from, city_to])

# First-level costs: producer -> warehouse
cost_p_to_w = {
    (p, w): distance(p, w) for p, w in product(producers, warehouses)
}

# Second-level costs: warehouse -> customer
cost_w_to_c = {
    (w, c): distance(w, c) for w, c in product(warehouses, customers)
}

print("First-level cost matrix (producer -> warehouse):")
for p in producers:
    row = ", ".join(f"{p} -> {w}: {cost_p_to_w[(p,w)]:.0f}" for w in warehouses)
    print(row)

print("\nSecond-level cost matrix (warehouse -> customer):")
for w in warehouses:
    row = ", ".join(f"{w} -> {c}: {cost_w_to_c[(w,c)]:.0f}" for c in customers)
    print(row)


First-level cost matrix (producer -> warehouse):
Liberec -> Kladno: 135, Liberec -> Pardubice: 155
Olomouc -> Kladno: 314, Olomouc -> Pardubice: 135
Plzeň -> Kladno: 100, Plzeň -> Pardubice: 213

Second-level cost matrix (warehouse -> customer):
Kladno -> Brno: 240, Kladno -> Praha: 31, Kladno -> Ostrava: 406, Kladno -> Zlín: 332
Pardubice -> Brno: 146, Pardubice -> Praha: 121, Pardubice -> Ostrava: 234, Pardubice -> Zlín: 201


## Krok 2 – Matematický model dvouúrovňového dopravního problému

### 2.1 Proměnné

Zavedeme dvě sady rozhodovacích proměnných:

- $y_{pk} \ge 0$ – množství zboží přepravené z **producenta** $p$ do **skladu** $k$,
- $x_{kj} \ge 0$ – množství zboží přepravené ze **skladu** $k$ k **odběrateli** $j$.

### 2.2 Parametry

- $s_p$ – kapacita (maximální produkce) producenta $p$,
- $b_j$ – požadavek odběratele $j$,
- $u_k$ – kapacita skladu $k$,
- $c^{(1)}_{pk}$ – náklady na jednotku zboží z producenta $p$ do skladu $k$,
- $c^{(2)}_{kj}$ – náklady na jednotku zboží ze skladu $k$ k odběrateli $j$.

### 2.3 Účelová funkce

Chceme minimalizovat **součet nákladů na obou úrovních**:

$$
\min z =
\sum_{p \in P} \sum_{k \in K} c^{(1)}_{pk} y_{pk} +
\sum_{k \in K} \sum_{j \in J} c^{(2)}_{kj} x_{kj}.
$$

### 2.4 Omezení

1. **Kapacita producentů**

Každý producent nesmí poslat více, než je jeho kapacita:

$$
\sum_{k \in K} y_{pk} \le s_p, \quad \forall p \in P.
$$

2. **Zachování toku ve skladech**

Co do skladu přiteče, to musí také odtéct (neuvažujeme skladování do zásoby):

$$
\sum_{p \in P} y_{pk} = \sum_{j \in J} x_{kj}, \quad \forall k \in K.
$$

3. **Kapacita skladů**

Celkový tok přes sklad nesmí překročit jeho kapacitu:

$$
\sum_{j \in J} x_{kj} \le u_k, \quad \forall k \in K.
$$

4. **Požadavky odběratelů**

Každý odběratel musí dostat alespoň své požadované množství:

$$
\sum_{k \in K} x_{kj} \ge b_j, \quad \forall j \in J.
$$

5. **Nezápornost**

$$
y_{pk} \ge 0, \quad x_{kj} \ge 0 \quad \text{pro všechny } p,k,j.
$$

Tento model nyní převedeme na PuLP model.


In [ ]:
# -----------------------------
# Step 2: Build the two-level LP model in PuLP
# -----------------------------

# Create minimization problem
model = LpProblem(name="Dvouurovnovy_dopravni_problem", sense=LpMinimize)

# First-level decision variables: producer -> warehouse
# y[p][w] = amount shipped from producer p to warehouse w
y = LpVariable.dicts(
    name="y",
    indices=(producers, warehouses),
    lowBound=0,
    cat="Continuous",
)

# Second-level decision variables: warehouse -> customer
# x[w][c] = amount shipped from warehouse w to customer c
x = LpVariable.dicts(
    name="x",
    indices=(warehouses, customers),
    lowBound=0,
    cat="Continuous",
)

# Objective function: sum of first- and second-level transport costs
model += (
    lpSum(y[p][w] * cost_p_to_w[(p, w)] for p in producers for w in warehouses)
    + lpSum(x[w][c] * cost_w_to_c[(w, c)] for w in warehouses for c in customers),
    "Total_transport_cost",
)

# 1) Producer capacity constraints
for p in producers:
    model += (
        lpSum(y[p][w] for w in warehouses) <= producer_supply[p],
        f"Producer_capacity_{p}",
    )

# 2) Flow conservation at each warehouse (incoming = outgoing)
for w in warehouses:
    model += (
        lpSum(y[p][w] for p in producers)
        == lpSum(x[w][c] for c in customers),
        f"Flow_balance_{w}",
    )

# 3) Warehouse capacity constraints
for w in warehouses:
    model += (
        lpSum(x[w][c] for c in customers) <= warehouse_capacity[w],
        f"Warehouse_capacity_{w}",
    )

# 4) Customer demand constraints
for c in customers:
    model += (
        lpSum(x[w][c] for w in warehouses) >= customer_demand[c],
        f"Customer_demand_{c}",
    )

print(model)


Dvouurovnovy_dopravni_problem:
MINIMIZE
240.0*x_Kladno_Brno + 406.0*x_Kladno_Ostrava + 31.0*x_Kladno_Praha + 332.0*x_Kladno_Zlín + 146.0*x_Pardubice_Brno + 234.0*x_Pardubice_Ostrava + 121.0*x_Pardubice_Praha + 201.0*x_Pardubice_Zlín + 135.0*y_Liberec_Kladno + 155.0*y_Liberec_Pardubice + 314.0*y_Olomouc_Kladno + 135.0*y_Olomouc_Pardubice + 100.0*y_Plzeň_Kladno + 213.0*y_Plzeň_Pardubice + 0.0
SUBJECT TO
Producer_capacity_Liberec: y_Liberec_Kladno + y_Liberec_Pardubice <= 6000

Producer_capacity_Olomouc: y_Olomouc_Kladno + y_Olomouc_Pardubice <= 2000

Producer_capacity_Plzeň: y_Plzeň_Kladno + y_Plzeň_Pardubice <= 3000

Flow_balance_Kladno: - x_Kladno_Brno - x_Kladno_Ostrava - x_Kladno_Praha
 - x_Kladno_Zlín + y_Liberec_Kladno + y_Olomouc_Kladno + y_Plzeň_Kladno = 0

Flow_balance_Pardubice: - x_Pardubice_Brno - x_Pardubice_Ostrava
 - x_Pardubice_Praha - x_Pardubice_Zlín + y_Liberec_Pardubice
 + y_Olomouc_Pardubice + y_Plzeň_Pardubice = 0

Warehouse_capacity_Kladno: x_Kladno_Brno + x_Kladno

## Krok 3 – Řešení modelu a zobrazení výsledků

Teď model předáme řešiči LP (opět pomocí PuLP) a zjistíme:

- zda bylo nalezeno optimální řešení,
- jaká je minimální hodnota účelové funkce (celkové náklady),
- jaké toky zboží probíhají:
  - z producentů do jednotlivých skladů,
  - ze skladů k odběratelům.

Pro lepší kontrolu také ověříme:

- že každý producent využívá nejvýše svou kapacitu,
- že každý sklad splňuje rovnováhu toku i kapacitu,
- že každý odběratel má splněný požadavek.


In [ ]:
# -----------------------------
# Step 3: Solve the model and inspect the solution
# -----------------------------

# Solve the LP problem
solution_status = model.solve()

print(f"Solve status: {LpStatus[model.status]}")
print(f"Optimal total cost (sum of all distances): {value(model.objective):,.0f}")

# --- First-level flows: producer -> warehouse ---
print("\nOptimal shipments from producers to warehouses:")
for p in producers:
    for w in warehouses:
        qty = y[p][w].varValue
        if qty is not None and qty > 1e-6:
            print(
                f"{p:8s} -> {w:9s} : {qty:6.0f} units "
                f"(cost per unit = {cost_p_to_w[(p, w)]:.0f})"
            )

# --- Second-level flows: warehouse -> customer ---
print("\nOptimal shipments from warehouses to customers:")
for w in warehouses:
    for c in customers:
        qty = x[w][c].varValue
        if qty is not None and qty > 1e-6:
            print(
                f"{w:9s} -> {c:8s} : {qty:6.0f} units "
                f"(cost per unit = {cost_w_to_c[(w, c)]:.0f})"
            )

# --- Checks ---

print("\nProducer usage (sum of shipments from each producer):")
for p in producers:
    total_out = sum(y[p][w].varValue for w in warehouses)
    print(f"{p:8s}: sent {total_out:6.0f} / capacity {producer_supply[p]}")

print("\nWarehouse balances and capacities:")
for w in warehouses:
    inbound = sum(y[p][w].varValue for p in producers)
    outbound = sum(x[w][c].varValue for c in customers)
    print(
        f"{w:9s}: inbound {inbound:6.0f}, outbound {outbound:6.0f}, "
        f"capacity {warehouse_capacity[w]}"
    )

print("\nCustomer demand satisfaction:")
for c in customers:
    total_in = sum(x[w][c].varValue for w in warehouses)
    print(f"{c:8s}: received {total_in:6.0f} / demand {customer_demand[c]}")


Solve status: Optimal
Optimal total cost (sum of all distances): 2,722,000

Optimal shipments from producers to warehouses:
Liberec  -> Kladno    :   2000 units (cost per unit = 135)
Liberec  -> Pardubice :   4000 units (cost per unit = 155)
Olomouc  -> Pardubice :   2000 units (cost per unit = 135)
Plzeň    -> Kladno    :   3000 units (cost per unit = 100)

Optimal shipments from warehouses to customers:
Kladno    -> Praha    :   5000 units (cost per unit = 31)
Pardubice -> Brno     :   3000 units (cost per unit = 146)
Pardubice -> Ostrava  :   2000 units (cost per unit = 234)
Pardubice -> Zlín     :   1000 units (cost per unit = 201)

Producer usage (sum of shipments from each producer):
Liberec : sent   6000 / capacity 6000
Olomouc : sent   2000 / capacity 2000
Plzeň   : sent   3000 / capacity 3000

Warehouse balances and capacities:
Kladno   : inbound   5000, outbound   5000, capacity 7000
Pardubice: inbound   6000, outbound   6000, capacity 7000

Customer demand satisfaction:
Brno

## Krok 4 – Interpretace řešení

Po spuštění předchozí buňky byste měli vidět (pro typickou verzi tabulky `cz.csv`) zhruba následující optimální plán:

### 4.1 Tok producent → sklad

- Liberec → Kladno: **2000** jednotek  
- Liberec → Pardubice: **4000** jednotek  
- Olomouc → Pardubice: **2000** jednotek  
- Plzeň → Kladno: **3000** jednotek  

Všechny ostatní kombinace producent–sklad mají tok 0.

Kontrola kapacit producentů:

- Liberec posílá $2000 + 4000 = 6000$ jednotek → plně využitá kapacita.  
- Olomouc posílá $2000$ jednotek → přesně kapacita.  
- Plzeň posílá $3000$ jednotek → přesně kapacita.  

### 4.2 Tok sklad → odběratel

- Kladno → Praha: **5000** jednotek  
- Pardubice → Brno: **3000** jednotek  
- Pardubice → Ostrava: **2000** jednotek  
- Pardubice → Zlín: **1000** jednotek  

Ostatní relace sklad–odběratel mají tok 0.

Kontrola:

- Sklad Kladno: přítok $2000 + 0 + 3000 = 5000$, odtok $5000$ → rovnováha toku, využití kapacity $5000 \le 7000$.  
- Sklad Pardubice: přítok $4000 + 2000 + 0 = 6000$, odtok $3000 + 2000 + 1000 = 6000$ → rovnováha toku, využití kapacity $6000 \le 7000$.  

### 4.3 Odběratelé

- Brno dostává $3000$ jednotek → splňuje požadavek 3000.  
- Praha dostává $5000$ jednotek → splňuje požadavek 5000.  
- Ostrava dostává $2000$ jednotek → splňuje požadavek 2000.  
- Zlín dostává $1000$ jednotek → splňuje požadavek 1000.  

### 4.4 Náklady

Hodnota účelové funkce (součet všech vzdáleností × množství) je přibližně

$$
z^\star = 2\,722\,000
$$

v jednotkách „kilometr–jednotka“. Pokud bychom měli konkrétní cenu za kilometr, stačí tuto hodnotu vynásobit danou cenou.
